In [1]:
import numpy as np
from scipy.optimize import linprog, minimize_scalar
from numpy.linalg import norm, inv, multi_dot
from numdifftools import Gradient

In [2]:
def f(x):
    return 0.5 * ((x[0] ** 2) + (x[1] ** 2))

A = np.array([[-1, 1], [1, 1], [0, -1]], dtype=float)
b = [7, 5, -2]

# Projected gradient algorithm

In [3]:
def projected_gradient(f, A, b, x0, tol, t=3):
    x = x0
    m = len(A)
    
    while True:
        grad = Gradient(f)(x)
        I = []
        for i in range(m):
            if abs(np.vdot(A[i], x) - b[i]) < tol:
                I.append(i)
        v = -multi_dot([inv(np.dot(A[I], A[I].T)), A[I], grad])
        d = -grad - np.dot(A[I].T, v)

        alpha_bar = np.inf
        for i in range(m):
            a = np.vdot(A[i], d)
            if (i not in I) and (a > 0):
                alpha = (b[i] - np.vdot(A[i], x)) / a
                alpha_bar = min(alpha_bar, alpha)

        phi = lambda alpha: f(x + alpha * d)
        alpha = minimize_scalar(phi, bounds=(0, alpha_bar), method='bounded').x
        x = x + alpha * d

        if norm(d) <= t:
            return x

In [4]:
x0 = [-2, 3]
x_sol = projected_gradient(f, A, b, x0, 0.0001)
print(x_sol)

[-4.44089210e-16  2.00001524e+00]
